In [1]:
import numpy as np
from tensorflow.keras.applications import resnet

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

In [3]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

In [4]:
train_data_dir = 'train'
test_data_dir = 'test'

In [5]:
from keras_preprocessing.image import ImageDataGenerator

In [6]:
img_rows, img_cols = 224, 224 
model = resnet.ResNet50(weights='imagenet', include_top = False, input_shape=(img_rows,img_cols,3))

In [80]:
for layer in model.layers:
    layer.trainable = False

In [82]:
#no trainable paramters in our model since we gonna use the pre trained weights
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [83]:
def addTopModel(bottom_model, num_classes, D=256):
    #creates the top or head of the model that will be placed ontop of the bottom layers

    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [84]:
num_classes=2
newmodel= addTopModel(model,num_classes)

In [85]:
modelnew = Model(inputs = model.input, outputs = newmodel)


In [86]:
modelnew.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

In [7]:
train_batchsize = 16

In [89]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')

Found 720 images belonging to 2 classes.


In [90]:
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

In [91]:
history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = 32,
    epochs = 3,
   )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 32 steps
Epoch 1/3
32/32 [==============================] - 178s 6s/step - loss: 15.1450 - accuracy: 0.8359
Epoch 2/3
32/32 [==============================] - 153s 5s/step - loss: 0.7204 - accuracy: 0.9570
Epoch 3/3
32/32 [==============================] - 153s 5s/step - loss: 0.5987 - accuracy: 0.9609


In [92]:
modelnew.save('facedetect.h5')

In [17]:
from matplotlib import pyplot as plt

In [18]:
plt.plot(history.history['accuracy'])
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Resnet")

NameError: name 'history' is not defined

In [10]:
test_generator = train_datagen.flow_from_directory(test_data_dir,(224,224))

Found 201 images belonging to 2 classes.


In [ ]:
model = keras.models.load_model('facedetect.h5')

In [15]:
yhat = model.predict_generator(test_generator, steps=24)

Instructions for updating:
Please use Model.predict, which supports generators.


In [16]:
result=[]
for i in range(len(yhat)):
    if(result[i][0]>0.5):
        result.append(1)
    else:
        result.append(0)

array([[1.0000000e+00, 4.2921449e-16],
       [1.0000000e+00, 4.1276368e-16],
       [1.0000000e+00, 2.4590134e-17],
       [1.0000000e+00, 3.7350310e-16],
       [1.0000000e+00, 1.4928688e-15],
       [1.0000000e+00, 2.6724239e-16],
       [1.0000000e+00, 3.6756383e-16],
       [1.0000000e+00, 6.1259468e-17],
       [1.0000000e+00, 2.1916541e-15],
       [1.0000000e+00, 1.2174822e-15],
       [1.0000000e+00, 1.9670381e-15],
       [1.0000000e+00, 1.4420449e-15],
       [1.0000000e+00, 4.5052216e-16],
       [1.0000000e+00, 6.2940144e-16],
       [1.0000000e+00, 1.3616598e-15],
       [1.0000000e+00, 4.2287620e-16],
       [1.0000000e+00, 1.2319369e-15],
       [1.0000000e+00, 3.9957165e-16],
       [1.0000000e+00, 1.4537416e-17],
       [1.0000000e+00, 5.2620768e-16],
       [1.0000000e+00, 3.2842191e-15],
       [1.0000000e+00, 2.4027127e-15],
       [1.0000000e+00, 4.3820122e-16],
       [1.0000000e+00, 3.5664686e-15],
       [1.0000000e+00, 4.3558322e-15],
       [1.0000000e+00, 1.

In [ ]:
result